In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive

/content/drive/MyDrive


In [3]:
!pip install transformers
!pip install sentencepiece
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 98.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 65.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 90.1 MB

In [4]:
!pip install bertopic[visualization]

In [5]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-oq7ohly3/kobert-tokenizer_d572b46c283d45a7bb63dbb29527c14a
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-oq7ohly3/kobert-tokenizer_d572b46c283d45a7bb63dbb29527c14a
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=00372bf88eae23d421c77405f0bbd117888acb4c5eec738f4a4de0de2fa97aba
  Stored in directory: /tmp/pip-ephem-wheel-cache-34raz9xw/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [6]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer.encode("한국어 모델을 공유합니다.")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

In [7]:
import torch
from transformers import BertModel
model = BertModel.from_pretrained('skt/kobert-base-v1')
text = "한국어 모델을 공유합니다."
inputs = tokenizer.batch_encode_plus([text])
out = model(input_ids = torch.tensor(inputs['input_ids']), attention_mask = torch.tensor(inputs['attention_mask']))
out.pooler_output.shape

torch.Size([1, 768])

In [8]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [9]:
inputs = tokenizer.batch_encode_plus([text], return_tensors='pt')
model_output = model(**inputs)
mean_pooling(model_output, inputs['attention_mask'])

tensor([[ 3.6122e-02, -1.2337e-01, -7.5301e-03, -5.0620e+00,  1.6959e-01,
          9.2553e-02, -2.8838e-01,  2.1198e-01, -6.3860e-01, -3.6451e-01,
         -1.6556e-02,  1.8412e-01, -2.9757e-01,  5.1233e-02,  4.4390e-02,
          2.3413e-01,  3.8918e-01, -2.7811e-02,  2.5196e-01, -1.6740e-01,
         -5.0612e-02, -1.1393e-01, -5.8496e-02, -9.2304e-02, -3.3941e-01,
          2.9963e-02, -6.9979e-02, -1.2118e-01, -1.7573e-01,  2.4531e-01,
          1.6764e-01,  6.0146e-02, -1.3626e-01,  1.6598e-01,  1.2201e-01,
          5.2110e-02, -3.8716e-02, -3.4097e-02,  2.7250e-01, -6.6029e-02,
          2.2798e-01,  2.6542e-02,  3.8524e-01, -5.3627e-02,  9.2007e-02,
          1.2872e-01, -9.4732e-02,  1.4018e-01,  4.4361e-02,  3.0786e-02,
         -2.6999e-01,  2.5637e-02,  5.1933e-02,  1.9702e-01, -1.4610e-01,
         -2.9428e-01,  1.8972e-01, -1.1122e-01, -2.5106e-01,  1.7953e-01,
          1.1555e-01, -1.8627e-01,  1.9638e-02, -1.5012e-01,  6.5611e-02,
          6.9550e-02,  1.3693e-01,  3.

In [10]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
import os
import json

In [11]:
def get_data(gallery):
  result = []
  for name in os.listdir(path='/content/drive/MyDrive/joinon/' + gallery):
    with open('/content/drive/MyDrive/joinon/' + gallery + '/' + name, 'r', encoding='utf-8') as f:
      data = json.load(f)
      valid = []
      for d in data['content'].split('\n'):
        d = d.replace('- dc official App', '').strip()
        if d and not d.replace(' ', '').isdecimal():
          valid.append(d)
      result.append('\n'.join([data['title']] + valid))
  return result

In [12]:
import random
cocktail = random.sample(get_data('cock_tail'),195) + random.sample(get_data('brandy'),195) + random.sample(get_data('ulisul'),195) + random.sample(get_data('wine'),195)
cocktail[:5]

['릴레이 시작하자구. 나부터야\n이미지 순서 ON\n어을패',
 '조주기능사 빈자리 신청 가능할려나...\n이게 신청기한이 은근 빡세네...\n애초에 이런거 1도 안봐서 몰랐음\n그래서 빈자리 신청 기한도 있는데\n오픈하자 마자 신청하면 가능할려나?',
 '나눔릴) 진토니꾸 받고\n코폴',
 '나눔릴) 흑백받고\n럼콬',
 '홈바3\n좀---비']

In [13]:
class CustomTokenizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
  def __call__(self, target):
    return self.tokenizer.tokenize(target)

In [14]:
custom_tokenizer = CustomTokenizer(tokenizer)
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [15]:
# bertopic_model = BERTopic(embedding_model=model,
#                  vectorizer_model=vectorizer,
#                  nr_topics=50,
#                  top_n_words=10,
#                  calculate_probabilities=True)
bertopic_model = BERTopic(language='multilingual',
                          nr_topics=50,
                          top_n_words=10,
                          calculate_probabilities=True)

In [16]:
topics, probs = bertopic_model.fit_transform(cocktail)

In [17]:
bertopic_model.visualize_topics()

In [18]:
bertopic_model.visualize_distribution(probs[0])

In [19]:
for i in range(0, 50):
  print(i,'번째 토픽 :', bertopic_model.get_topic(i))

0 번째 토픽 : [('the', 0.050367981913392354), ('쌀전분', 0.030637035544940253), ('https', 0.029637761122669127), ('같다', 0.02421465405571248), ('of', 0.023894915312728644), ('starter', 0.02256683618635503), ('culture', 0.020445875832885974), ('ec', 0.020445875832885974), ('kr', 0.018610702349355718), ('in', 0.018273608527941316)]
1 번째 토픽 : [('이야', 0.026859478351194854), ('가격', 0.024421794377250644), ('가격은', 0.02391624299703438), ('and', 0.019892986247421546), ('가격이', 0.019710954461730115), ('상당히', 0.014409753256667202), ('그냥', 0.01427427098307708), ('와인', 0.013050351021639954), ('그리고', 0.012718919604887786), ('보르도', 0.0127173219590963)]
2 번째 토픽 : [('와인', 0.01946271806983788), ('같은', 0.012743703180797388), ('너무', 0.012683518565620498), ('느낌이', 0.012021429411314195), ('맛이', 0.011804481353829279), ('향이', 0.0117179436658667), ('이런', 0.01145859961834945), ('다른', 0.011139430036748759), ('만든', 0.010725504753016467), ('있는', 0.010475056639871921)]
3 번째 토픽 : [('모스카토', 0.04284145633145432), ('이거', 0.0229

In [20]:
for i in range(8):
  print(topics.count(i))

21
91
177
91
10
133
0
0


In [21]:
print(topics)

[-1, -1, 5, 5, 5, 5, -1, 2, -1, 3, 3, 5, 5, -1, 3, 3, 5, 2, -1, -1, 3, 5, 3, 3, -1, 5, 5, -1, -1, -1, -1, 5, 2, 3, 5, -1, 2, 5, 1, 2, 5, 2, 5, 3, 1, 5, -1, 3, 5, -1, 5, 5, 2, 5, 5, -1, 2, 1, 3, 5, -1, 5, 5, -1, 2, 3, 3, 5, -1, -1, 2, 5, 3, -1, 2, -1, -1, 5, -1, 5, -1, -1, 5, 5, 2, 3, 5, 5, -1, 5, -1, 5, 2, 2, 5, 5, 5, 3, -1, 5, -1, 5, 2, 5, 3, -1, 5, -1, 1, 0, 3, 2, -1, 4, 2, 3, 5, -1, 5, -1, 5, 5, -1, 4, 5, 3, 5, 1, 4, 5, -1, -1, 5, 5, -1, -1, -1, 5, 5, 3, 2, -1, 3, -1, 2, 5, -1, 5, 5, -1, -1, 3, 5, 5, 5, 1, 2, 2, 5, 5, -1, -1, 5, 1, 3, -1, 3, 4, 2, 5, 5, -1, 5, 2, 3, -1, 3, 5, 1, 5, 5, 5, 5, 5, 5, 5, -1, 5, 5, -1, -1, 5, -1, -1, -1, -1, -1, -1, 5, 3, 1, 5, 1, 2, 2, 3, -1, 3, 5, 3, 5, -1, 1, 2, 2, -1, -1, -1, -1, 2, 1, -1, -1, 1, -1, 3, 2, 2, -1, 1, 2, -1, 2, 3, 1, -1, -1, 5, 3, 5, 1, 3, 1, 5, 2, 1, 3, 2, -1, 1, -1, 3, 2, 3, 4, 2, -1, 3, 2, 2, 1, -1, -1, -1, 1, -1, 3, -1, 2, 1, 2, 1, -1, 1, 5, -1, 3, 1, 2, 3, -1, -1, 1, 2, 5, 1, 5, -1, 5, -1, 5, -1, -1, 5, -1, 0, 3, -1, 5, -1, -1, -1,

In [22]:
from sklearn.cluster import KMeans

# 우리가 원하는 클러스터의 수를 지정합니다.
n_clusters = 4

# KMeans 객체를 생성합니다.
kmeans = KMeans(n_clusters=n_clusters, random_state=0)

# 확률 데이터를 사용하여 KMeans 클러스터링을 수행합니다.
kmeans.fit(probs)

# 각 데이터 포인트가 속한 클러스터의 레이블을 얻습니다.
labels = kmeans.labels_


In [23]:
# 실제 레이블 생성
real_labels = [0]*195 + [1]*195 + [2]*195 + [3]*195

# 실제 레이블과 클러스터 레이블을 비교
from sklearn.metrics import confusion_matrix
print(confusion_matrix(real_labels, labels))


[[ 23 101  63   8]
 [ 35 114  15  31]
 [ 45 120  22   8]
 [ 67  89   5  34]]
